## 1. Import Libraries and Load Dataset File after Preprocessing and Labelling

In [52]:
"""
Meng-import library yang akan dibutuhkan untuk melakukan proses
pemodelan dengan 2 Algoritma Machine Learning: Naive Bayes dan Support Vector Machine (SVM)
"""
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("ggplot")
import seaborn as sns
from wordcloud import WordCloud
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB # Library untuk model naive bayes yang dikhususkan untuk memproses data teks
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

In [53]:
"""
Membaca file dataset setelah dilakukan proses pelabelan dengan 3 nilai label sentimen (positif, negatif dan netral)
dan juga telah dilakukan text processing untuk seluruh data komentar yang berhasil diambil dari video debat
babak pertama hingga kelima capres - cawapres tahun 2024 yang di-upload/ditayangkan melalui 3 channel YouTube yakni
Najwa Shihab, KOMPASTV dan KPU RI
"""
youtube_comments_df = pd.read_excel("Dataset/Text Processing and Labeling Results Dataset.xlsx")

youtube_comments_df.head(9) # Menampilkan 9 data teratas dari variabel DataFrame "youtube_comments_df"

video_id                channel_id                  channel_name  \
0  Sbjsg3AFx00  UCo8h2TY_uBkAVUIc14m_KCA  Najwa Shihab YouTube Channel   
1  gUz_MgdwKg0  UCo8h2TY_uBkAVUIc14m_KCA  Najwa Shihab YouTube Channel   
2  tEbRB5xxOPI  UCo8h2TY_uBkAVUIc14m_KCA  Najwa Shihab YouTube Channel   
3  tEbRB5xxOPI  UCo8h2TY_uBkAVUIc14m_KCA  Najwa Shihab YouTube Channel   
4  KJdt-HBBGIo  UCVdSz86o9q2cQKvAQ5QhOYg        KPU RI YouTube Channel   
5  gUz_MgdwKg0  UCo8h2TY_uBkAVUIc14m_KCA  Najwa Shihab YouTube Channel   
6  gUz_MgdwKg0  UCo8h2TY_uBkAVUIc14m_KCA  Najwa Shihab YouTube Channel   
7  JFl2fOTBNIE  UCo8h2TY_uBkAVUIc14m_KCA  Najwa Shihab YouTube Channel   
8  gUz_MgdwKg0  UCo8h2TY_uBkAVUIc14m_KCA  Najwa Shihab YouTube Channel   

                   comment_id                    author  \
0  Ugx6mhUfDOUhd65fWyh4AaABAg          @udinlamidun9716   
1  UgxfMHgX25B2b3jkO4B4AaABAg            @zakishofi3442   
2  UgwTpSK1hjkTNQlcT894AaABAg            @sandigustiana   
3  UgzaWyuIp8PPLQrUOct4AaABAg         @abizardfahry4591   
4  Ugy7bfVbuDxBgae9eVp4AaABAg          @nabilbudiman271   
5  UgzKC1PcDJZk7TvQ_Qh4AaABAg           @UmmuAzizah2010   
6  Ugxtzzc3zKJ8nY3Sg7F4AaABAg            @nursusilo7298   
7  UgwB34PptWoq1w3z7xN4AaABAg  @lilissuryanilimbong7685   
8  Ugypp7MrvLOhoz9GGUV4AaABAg           @abisopiyan8628   

                                  dirty_text_comment  \
0  Siapa yg setuju 3 capres di undang bertiga di ...   
1  Saya bukan pendukung siapapun, tapi terimakasi...   
2  Timestamp<br><br><a href="https://www.youtube....   
3  Siapa yang disini setuju 3 capres dan cawapres...   
4  <a href="https://www.youtube.com/watch?v=KJdt-...   
5  Maa syaa Allaah Pak Anies ❤❤❤❤ <br>Terima kasi...   
6  Tak terbantahkan Pak Anis cards yg paling siap...   
7  Selaku Ibu Rumah Tangga saya tahu apa yg dibut...   
8  Saya dan keluarga berkat debat pertama ini ras...   

                                  clean_text_comment  like_count  total_reply  \
0  tuju capres undang tiga mata najwa debat nyata...        3970          332   
1  dukung terimakasih anies bicara kanjuruhan hre...        3182           91   
2  timestamp href https www youtube com watch teb...        2936          124   
3  tuju capres cawapres undang tiga mata najwa de...        2531           91   
4  href https www youtube com watch kjdt hbbgio a...        1590          122   
5  maa syaa allaah anies terima kasih cerah sbg r...        1542           23   
6  bantah anis cards logis sistematis educativ ko...        1501           91   
7  rumah tangga butuh anak makan gratis solusi ma...        1486          169   
8  keluarga berkat debat pilih anies baswedan pre...        1452           54   

       published_date        updated_date  subjectivity  polarity  \
0 2024-01-07 17:03:53 2024-01-09 23:18:49      0.300000  0.200000   
1 2023-12-12 21:22:08 2023-12-12 21:22:08      0.000000  0.000000   
2 2024-01-21 21:32:13 2024-01-21 21:32:13      0.000000  0.000000   
3 2024-01-22 00:17:18 2024-01-22 00:17:18      0.300000  0.200000   
4 2024-01-07 15:37:05 2024-01-07 15:37:05      0.474444  0.313333   
5 2023-12-12 16:27:52 2023-12-12 16:27:52      0.800000  0.700000   
6 2023-12-12 21:00:05 2023-12-12 21:00:05      1.000000  0.900000   
7 2024-02-04 20:29:41 2024-02-04 20:29:41      0.703611  0.368333   
8 2023-12-12 17:49:04 2023-12-12 17:49:04      0.200000  0.200000   

  sentiment_label  
0         positif  
1          netral  
2          netral  
3         positif  
4         positif  
5         positif  
6         positif  
7         positif  
8         positif

## 2. Exploratory Dataset after Finished Labelling for Sentiment and Text Processing

In [54]:
# Menampilkan hanya nilai yang ada di dalam masing-masing kolom tetapi kolom tidak diikutsertakan
youtube_comments_df.values

array([['Sbjsg3AFx00', 'UCo8h2TY_uBkAVUIc14m_KCA',
        'Najwa Shihab YouTube Channel', ..., 0.3, 0.2, 'positif'],
       ['gUz_MgdwKg0', 'UCo8h2TY_uBkAVUIc14m_KCA',
        'Najwa Shihab YouTube Channel', ..., 0.0, 0.0, 'netral'],
       ['tEbRB5xxOPI', 'UCo8h2TY_uBkAVUIc14m_KCA',
        'Najwa Shihab YouTube Channel', ..., 0.0, 0.0, 'netral'],
       ...,
       ['gUz_MgdwKg0', 'UCo8h2TY_uBkAVUIc14m_KCA',
        'Najwa Shihab YouTube Channel', ..., 0.9, -0.8, 'negatif'],
       ['gUz_MgdwKg0', 'UCo8h2TY_uBkAVUIc14m_KCA',
        'Najwa Shihab YouTube Channel', ..., 0.0, 0.0, 'netral'],
       ['8J66JxvmEzo', 'UCVdSz86o9q2cQKvAQ5QhOYg',
        'KPU RI YouTube Channel', ..., 0.0, 0.0, 'netral']], dtype=object)

In [55]:
# Menampilkan jumlah baris dan kolom yang saat ini dimiliki oleh variabel DataFrame "youtube_comments_df"
youtube_comments_df.shape

(48239, 14)

Hasil diatas menunjukkan bahwa saat ini variabel DataFrame "youtube_comments_df" memiliki 14 kolom dan 48.239 baris data. Penambahan jumlah kolom ini terjadi karena sebelumnya telah dilakukan proses penilaian subjektivitas dan polaritas untuk seluruh kalimat komentar serta hasil dari pemrosesan data teks komentar yang telah dilakukan proses removal punctuations, case folding, tokenization, removal stopwords dan stemming.

In [56]:
# Menampilkan nama-nama kolom yang dimiliki oleh variabel DataFrame "youtube_comments_df"
youtube_comments_df.columns

Index(['video_id', 'channel_id', 'channel_name', 'comment_id', 'author',
       'dirty_text_comment', 'clean_text_comment', 'like_count', 'total_reply',
       'published_date', 'updated_date', 'subjectivity', 'polarity',
       'sentiment_label'],
      dtype='object')

Saat ini variabel DataFrame "youtube_comments_df" memiliki 14 kolom diantaranya adalah kolom "video_id", "channel_id", "channel_name", "comment_id", "author", "dirty_text_comment", "clean_text_comment", "like_count", "total_reply", "published_date", "updated_date", "subjectivity", "polarity", "sentiment_label".

Khusus untuk kolom "clean_text_comment", "subjectivity", "polarity" dan "sentiment_label" adalah kolom yang dibuat saat melakukan proses text processing dan juga pelabelan dengan 3 nilai untuk mengklasifikasikan sentimen berdasarkan nilai/skor yang didapatkan pada kolom "polarity". Sedangkan kolom "subjectivity" merupakan kolom yang menunjukkan suatu kalimat komentar apakah bersifat subjektif atau objektif. Jika skor/nilai pada kalimat komentar memiliki nilai 0 maka dapat dipastikan jika komentar tersebut bersifat objektif sedangkan jika skor pada kalimat komentar memiliki nilai 1 maka komentar tersebut dapat dikategorikan sebagai komentar yang subjektif.

In [57]:
# Menampilkan tipe data yang dimiliki oleh masing-masing kolom dari variabel DataFrame "youtube_comments_df"
youtube_comments_df.dtypes

video_id                      object
channel_id                    object
channel_name                  object
comment_id                    object
author                        object
dirty_text_comment            object
clean_text_comment            object
like_count                     int64
total_reply                    int64
published_date        datetime64[ns]
updated_date          datetime64[ns]
subjectivity                 float64
polarity                     float64
sentiment_label               object
dtype: object

Berdasarkan hasil diatas dapat diketahui jika terdapat penambahan tipe data baru dari sebelumnya yaitu Float dan Datetime karena pada saat proses penggabungan pertama antara seluruh data komentar dari YouTube channel Najwa Shihab, KOMPASTV dan KPU RI, tidak memiliki tipe data Float atau tipe data Datetime. Perlu diketahui dan diingat kembali jika tipe data Float merepresentasikan bilangan angka numerik desimal yaitu sebuah angka pecahan sedangkan tipe data Integer merepresentasikan bilangan bulat yaitu sebuah angka tanpa adanya floating point. Sedangkan tipe data Datetime digunakan untuk merepresentasikan suatu format data yang menunjukkan informasi/keterangan tanggal dan waktu dari suatu kejadian.

Di samping itu, tipe data Object/String merupakan tipe data yang paling banyak dimiliki oleh kolom yang tersimpan di dalam variabel DataFrame "youtube_comments_df". Nama-nama kolom yang memiliki tipe data Object/String dari DataFrame "youtube_comments_df" adalah kolom "video_id", "channel_id", "channel_name", "comment_id", "author", "dirty_text_comment", "clean_text_comment", dan terakhir adalah kolom "sentiment_label". Sementara itu variabel DataFrame "youtube_comments_df" juga memiliki tipe data Datetime yang berfungsi sebagai representasi kolom yang menunjukkan stempel tanggal dan waktu seperti tanggal dan waktu pembelian suatu barang, tanggal dan waktu saat teks komentar dibuat oleh para penonton konten video di media sosial YouTube, dll.

In [58]:
"""
Menampilkan masing-masing kolom data dari variabel DataFrame
"youtube_comments_df" yang memiliki tipe data selain Object/String
"""
youtube_comments_df.select_dtypes(exclude="O")

like_count  total_reply      published_date        updated_date  \
0            3970          332 2024-01-07 17:03:53 2024-01-09 23:18:49   
1            3182           91 2023-12-12 21:22:08 2023-12-12 21:22:08   
2            2936          124 2024-01-21 21:32:13 2024-01-21 21:32:13   
3            2531           91 2024-01-22 00:17:18 2024-01-22 00:17:18   
4            1590          122 2024-01-07 15:37:05 2024-01-07 15:37:05   
...           ...          ...                 ...                 ...   
48234           0            0 2024-01-07 19:08:29 2024-01-07 19:08:29   
48235           0            0 2023-12-13 01:31:02 2023-12-13 01:31:02   
48236           0            0 2023-12-13 01:31:42 2023-12-13 01:31:42   
48237           0            0 2023-12-13 01:31:45 2023-12-13 01:31:45   
48238           0            0 2024-02-04 18:22:22 2024-02-04 18:22:22   

       subjectivity  polarity  
0          0.300000  0.200000  
1          0.000000  0.000000  
2          0.000000  0.000000  
3          0.300000  0.200000  
4          0.474444  0.313333  
...             ...       ...  
48234      0.000000  0.000000  
48235      0.000000 -0.100000  
48236      0.900000 -0.800000  
48237      0.000000  0.000000  
48238      0.000000  0.000000  

[48239 rows x 6 columns]

In [59]:
"""
Menampilka kolom data dari variabel DataFrame
"youtube_comments_df" yang hanya memiliki tipe data datetime
"""
youtube_comments_df.select_dtypes(include="datetime")

published_date        updated_date
0     2024-01-07 17:03:53 2024-01-09 23:18:49
1     2023-12-12 21:22:08 2023-12-12 21:22:08
2     2024-01-21 21:32:13 2024-01-21 21:32:13
3     2024-01-22 00:17:18 2024-01-22 00:17:18
4     2024-01-07 15:37:05 2024-01-07 15:37:05
...                   ...                 ...
48234 2024-01-07 19:08:29 2024-01-07 19:08:29
48235 2023-12-13 01:31:02 2023-12-13 01:31:02
48236 2023-12-13 01:31:42 2023-12-13 01:31:42
48237 2023-12-13 01:31:45 2023-12-13 01:31:45
48238 2024-02-04 18:22:22 2024-02-04 18:22:22

[48239 rows x 2 columns]

In [60]:
"""
Menampilkan nilai-nilai statistik deskriptif yang dimiliki oleh beberapa
kolom dengan nilai data numerik (bertipe data Integer/Float) pada variabel DataFrame "youtube_comments_df"
"""
youtube_comments_df.describe()

like_count   total_reply  subjectivity      polarity
count  48239.000000  48239.000000  48239.000000  48239.000000
mean       5.624453      0.474968      0.079868      0.105707
std       47.191377      3.682588      0.212832      0.274946
min        0.000000      0.000000      0.000000     -1.000000
25%        0.000000      0.000000      0.000000      0.000000
50%        0.000000      0.000000      0.000000      0.000000
75%        1.000000      0.000000      0.000000      0.216667
max     3970.000000    332.000000      1.000000      1.000000

Berdasarkan hasil diatas dapat diketahui nilai-nilai statistik dari keempat kolom yang memiliki format nilai data numerik dari variabel DataFrame "youtube_comments_df" yaitu kolom "like_count", "total_reply", "subjectivity" dan "polarity". Sedangkan kolom-kolom seperti "video_id", "channel_id", "channel_name", "comment_id", "author", "dirty_text_comment", "clean_text_comment", "published_date", "updated_date" dan "sentiment_label" memiliki format nilai data bukan numerik. Bahkan khusus untuk kolom "published_date" dan "updated_date" memiliki format data tanggal dan waktu karena menunjukkan tanggal dan waktu dari pengiriman setiap kalimat komentar yang dibuat oleh para penonton video debat capres - cawapres tahun 2024.


Alasan kenapa keempat kolom tersebut terlihat nilai statistik deskriptifnya sedangkan kolom "video_id", "channel_id", "channel_name", "comment_id", "author", "dirty_text_comment", "clean_text_comment", "published_date", "updated_date" dan "sentiment_label" tidak ada dikarenakan kesepuluh kolom tersebut memiliki format nilai data kategorikal dengan tipe data string (kecuali kolom "published_date" dan "updated_date" yang merupakan data yang memiliki format Datetime). Sehingga dengan output diatas yang menunjukkan nilai-nilai statistik deskriptif, kita dapat mengetahui bahwasanya nilai count, mean (rata-rata), std (standar deviasi), min (minimum), 25% yang merupakan nilai quartil pertama, 50% yang menunjukkan sebuah nilai quartil kedua atau juga dapat dikategorikan sebagai nilai median, 75% yang merupakan nilai quartil ketiga dan nilai max (maximum) merupakan nilai statistik deskriptif untuk nilai data numerik yang memiliki tipe data Integer/Float. Jika kita ingin melihat sekaligus menampilkan nilai-nilai statistik deskriptif untuk kolom yang memiliki format nilai data kategorikal, kita dapat menggunakan 2 cara yang paling umum digunakan menggunakan library Pandas yaitu sebagai berikut:

- Cara pertama adalah dengan cara "nama variabel DataFrame.describe(include="O")". Nilai "O" pada parameter include di dalam fungsi .describe() merepresentasikan Object yaitu tipe data String untuk setiap nilai data yang memiliki format nilai kategorikal

- Cara kedua yaitu dapat menggunakan "nama variabel DataFrame.describe(include="all")". Namun jika menggunakan cara kedua maka seluruh kolom yang terdapat di dalam variabel DataFrame akan menampilkan nilai statistik deskriptifnya masing-masing yang disesuaikan dengan format nilai datanya.

Untuk mengetahui informasi yang terdapat pada output diatas yang menampilkan nilai statistik deskriptif untuk seluruh nilai data bertipe data numerik, berikut adalah informasi-informasi dan insight yang dapat diambil dari kolom "polarity" dan kolom "like_count" yang memiliki format nilai data numerik dengan tipe data Float dan Integer:

1. Kolom "polarity" memiliki nilai minimum yakni -1.0 dengan nilai quartil pertama dan kedua adalah 0.0. Hal ini cukup selaras dengan hasil pelabelan dimana untuk membuat label sentimen, setiap nilai dibawah angka 0 (angka minus) akan memiliki label Negatif. Selain itu nilai maksimum/nilai terbesar pada kolom "polarity" adalah angka 1.0. Sama seperti logika untuk angka  -1.0, setiap nilai angka polaritas yang melebihi angka 0, maka akan memiliki label sentimen Positif sehingga nilai terbesar yang dimiliki oleh kolom ini adalah nilai 1.0. Sedangkan nilai rata-rata pada kolom ini adalah 0.1 dengan standar deviasi adalah 0.3 dari angka 0.274946.

2. Kolom "like_count" memiliki nilai minimum yakni 0. Hal ini mengartikan jika tidak semua kalimat komentar yang terdapat pada masing-masing video debat capres - cawapres yang ditayangkan oleh channel YouTube Najwa Shihab, KOMPASTV dan KPU RI disetujui/disukai oleh para user atau akun YouTube yang menonton debat secara live streaming maupun rekaman. Di samping itu terdapat insight dimana nilai maksimum yang dimiliki oleh kolom "like_count" adalah 3.970. Hal ini menjelaskan bahwasanya terdapat kalimat komentar yang sangat disukai dan disetujui oleh beberapa akun/channel YouTube yang melihat teks komentar tersebut. Komentar yang memiliki jumlah disukai sebanyak 3.970 dibuat oleh akun/channel YouTube bernama @udinlamidun9716 membuat kalimat komentar seperti, "Siapa yg setuju 3 capres di undang bertiga di mata najwa untuk berdebat secara nyata dan menjawab dengan cukup waktu?? Perdebatan disini yg terlihat menonjol hanya pak anies saja soalnya.<br>MATA NAJWA, TOLONG BANTU AGAR KE 3 CAPRES BISA HADIR. AGAR KITA SEMUA BISA MELIHAT APAKAH PRABOWO BISA MENJAWAB KETIKA DI BERI WAKTU YG CUKUP?? JANGAN PAS DI DEBAT KPU NANGIS, TAPI PAS DI KANDANG MALAH TERIAK NGATA NGATAIN ANIES". Isi komentar tersebut dapat diketahui dan dilihat pada kolom "dirty_text_comment".

In [61]:
# Menampilkan nilai statistik deskriptif untuk kolom-kolom dengan tipe data String/Object
youtube_comments_df.describe(include="O")

video_id                channel_id                  channel_name  \
count         48239                     48239                         48239   
unique           15                         3                             3   
top     zB0RvKAc2kw  UCo8h2TY_uBkAVUIc14m_KCA  Najwa Shihab YouTube Channel   
freq          10500                     22999                         22999   

                        comment_id                 author  \
count                        48239                  48239   
unique                       48239                  36170   
top     Ugx6mhUfDOUhd65fWyh4AaABAg  @trianitarahmawati664   
freq                             1                     58   

                                      dirty_text_comment clean_text_comment  \
count                                              48239              47036   
unique                                             45662              41922   
top     GASPOLL BRO! PRABOWO GIBRAN UNTUK INDONESIA MAJU               amin   
freq                                                 177                403   

       sentiment_label  
count            48239  
unique               3  
top             netral  
freq             24587

Hasil diatas menunjukkan nilai-nilai statistik deskriptif untuk kolom-kolom yang memiliki tipe data String/Object. Tipe data String merupakan sebuah tipe data yang merepresentasikan nilai data berformat kategorikal yaitu sebuah data yang terdiri dari serangkaian huruf yang membentuk sebuah kata/kalimat, karakter spesial, simbol-simbol tanpa adanya angka. Di samping itu, nilai statistik desktriptif untuk kolom yang memiliki tipe data String/Object sangat berbeda dengan nilai statistik deskriptif untuk kolom yang memiliki nilai numerik seperti tipe data Integer dengan angka bilangan bulat dan tipe data Float dengan angka bilangan desimal. Jika pada nilai statistik deskriptif untuk kolom yang memiliki nilai data numerik/angka maka statistik deskriptifnya adalah nilai mean/rata-rata, standar deviasi, minimum, maksimum, quartil pertama (25%), quartil kedua atau yang biasa dikenal sebagai nilai median (50%) dan quartil ketiga (75%). Sedangkan nilai statistik deskriptif untuk kolom yang memiliki nilai data kategorikal adalah nilai unique, top dan freq atau frekuensi dari kemunculan data.

Nilai unique pada statistik deskriptif untuk kolom bertipe data String merupakan sebuah nilai yang memiliki keunikan tersendiri dan keunikan tersebut yang membuat satu data dengan data lainnya tidak sama. Sebagai contoh nilai unik yang dapat diketahui berdasarkan output diatas adalah kolom "channel_id" dan "channel_name" yang hanya memiliki 3 nilai saja yaitu Najwa Shihab YouTube Channel, KOMPAS TV YouTube Channel dan KPU RI YouTube Channel. Sedangkan kolom "channel_id" merupakan sebuah ID yang dimiliki oleh masing-masing tiga channel YouTube yaitu YouTube channel Najwa Shihab dengan ID-nya yaitu UCo8h2TY_uBkAVUIc14m_KCA, KOMPASTV yang memiliki ID channel yaitu UC5BMIWZe9isJXLZZWPWvBlg dan YouTube channel KPU RI dengan ID-nya adalah UCVdSz86o9q2cQKvAQ5QhOYg).


Nilai statistik deskriptif selanjutnya yang terdapat pada kolom bertipe data String/Object adalah nilai statistik top yang menunjukkan/merepresentasikan suatu nilai data yang paling banyak muncul di dalam kolom suatu kolom sehingga nilai tersebut paling menonjol diantara nilai-nilai data yang lainnya yang juga terdapat di kolom tersebut. Sebagai contoh nilai statistik top pada kolom "channel_name" adalah Najwa Shihab YouTube Channel karena nilai tersebut memiliki jumlah frekuensi kemunculan sebanyak 22.999 kali atau dapat diartikan jika nilai Najwa Shihab YouTube Channel merupakan nilai data yang paling sering terlihat di dalam kolom "channel_name". Untuk menampilkan jumlah frekuensi terbanyak di dalam suatu data yang dimiliki oleh kolom di dalam DataFrame, kita dapat menggunakan fungsi "DataFrame_Name.value_counts()".

Maka dari itu berlanjut kepada nilai statistik deskriptif selanjutnya yaitu nilai statistik freq yang merupakan jumlah total kemunculan/frekuensi suatu nilai data di dalam kolom. Nilai freq pada statistik deskriptif juga menunjukkan data tersebutlah yang paling sering muncul di dalam suatu kolom. Contoh lainnya adalah pada kolom "author", nilai statistik top di kolom "author" adalah username/akun channel YouTube bernama @trianitarahmawati664 yang memiliki jumlah kemunculan data terbanyak yaitu 58 kali di dalam variabel DataFrame "youtube_comments_df". Hal tersebut juga dapat mengartikan jika username @trianitarahmawati664 adalah username/akun yang paling sering membuat komentar di dalam video debat capres - cawapres tahun 2024 (setidaknya pada tiga channel YouTube yang berbeda dan keterbatasan dalam jumlah pengambilan nilai data dari masing-masing video debat).

In [62]:
# Menampilkan jumlah data unik yang terdapat pada masing-masing kolom di dalam variabel DataFrame "youtube_comments_df"
youtube_comments_df.nunique()

video_id                 15
channel_id                3
channel_name              3
comment_id            48239
author                36170
dirty_text_comment    45662
clean_text_comment    41922
like_count              381
total_reply              77
published_date        46573
updated_date          46579
subjectivity            828
polarity               2310
sentiment_label           3
dtype: int64

Hasil diatas menunjukkan jumlah keunikan data yang dimiliki oleh masing-masing kolom. Keunikan data merupakan suatu nilai data yang memiliki perbedaan antara satu nilai dengan nilai lainnya. Sebagai contoh pada kolom "sentiment_label" memiliki 3 data unik yaitu nilai sentiment positif, netral dan negatif. Kolom yang memiliki 3 nilai unik tidak hanya dimiliki oleh kolom "sentiment_label" karena kolom "channel_id" dan "channel_name" juga memiliki jumlah data unik yang sama yaitu 3 di mana nilai UCo8h2TY_uBkAVUIc14m_KCA yang merupakan ID dari channel YouTube Najwa Shihab, UCVdSz86o9q2cQKvAQ5QhOYg yang merupakan ID dari channel YouTube KPU RI dan terakhir adalah ID UC5BMIWZe9isJXLZZWPWvBlg dari YouTube channel KOMPASTV.

Di samping itu, kita juga dapat mengambil kesimpulan bahwasanya setiap teks komentar yang dibuat oleh masing-masing akun channel yang juga ikut menonton dan berkomentar di dalam tayangan video debat capres - cawapres tahun 2024 di channel YouTube Najwa Shihab, KOMPASTV dan KPU RI akan memiliki ID komentar yang berbeda-beda walaupun kalimat komentar yang dibuat adalah sama. Hal ini dapat diketahui sebab jumlah keunikan nilai data dari kolom "dirty_text_comment" (sebelum seluruh teks komentar diproses dengan Algoritma NLP) adalah 45.662 sedangkan jumlah nilai unik pada kolom "comment_id" adalah 48.239 data atau sama dengan jumlah baris data yang terdapat di dalam variabel DataFrame "youtube_comments_df". Informasi ini mengartikan walaupun terdapat teks komentar yang ditulis dengan isi dan susunan kata dan kalimat yang sama tetapi tetap akan memiliki ID yang berbeda. Hal ini juga selaras dengan prinsip ID secara general bahwa ID merupakan identitas unik dan harus berbeda antara satu nilai dengan nilai data lainnya walaupun nilai data tersebut adalah sama.

Eksplorasi selanjutnya adalah saya akan menampilkan jumlah frekuensi/kemunculan suatu nilai data khususnya kolom data yang memiliki tipe data String (data yang dibuat menggunakan format kategorikal).

In [63]:
# Menampilkan jumlah frekuensi/kemunculan suatu nilai data di dalam beberapa kolom dari variabel DataFrame "youtube_comments_df"
print("Jumlah Frekuensi/Kemunculan Data Terbanyak pada Kolom ID dari Ketiga YouTube Channel:")
print(youtube_comments_df["channel_id"].value_counts(), "\n")

print("Jumlah Frekuensi/Kemunculan Data Terbanyak pada Kolom Nama YouTube Channel:")
print(youtube_comments_df["channel_name"].value_counts(), "\n")

print("Jumlah Frekuensi/Kemunculan Data Terbanyak pada Kolom Author \n"
      "(Channel YouTube yang ikut Berkomentar di dalam Video Tayangan Debat):")
print(youtube_comments_df["author"].value_counts(), "\n")

print("Jumlah Frekuensi/Kemunculan Data Terbanyak pada Kolom Teks Komentar yang Sudah Diproses dengan NLTK:")
print(youtube_comments_df["clean_text_comment"].value_counts(), "\n")

print("Jumlah Frekuensi/Kemunculan Data Terbanyak pada Kolom Label Sentimen:")
print(youtube_comments_df["sentiment_label"].value_counts())

Jumlah Frekuensi/Kemunculan Data Terbanyak pada Kolom ID dari Ketiga YouTube Channel:
UCo8h2TY_uBkAVUIc14m_KCA    22999
UCVdSz86o9q2cQKvAQ5QhOYg    12740
UC5BMIWZe9isJXLZZWPWvBlg    12500
Name: channel_id, dtype: int64 

Jumlah Frekuensi/Kemunculan Data Terbanyak pada Kolom Nama YouTube Channel:
Najwa Shihab YouTube Channel    22999
KPU RI YouTube Channel          12740
KOMPAS TV YouTube Channel       12500
Name: channel_name, dtype: int64 

Jumlah Frekuensi/Kemunculan Data Terbanyak pada Kolom Author 
(Channel YouTube yang ikut Berkomentar di dalam Video Tayangan Debat):
@trianitarahmawati664        58
@rinideh6414                 56
@lasmiminah7171              52
@rzkadity69                  49
@dinarikaanggidesylia1408    49
                             ..
@sileasvirgil8393             1
@NadiraArsyilaTV              1
@yances6948                   1
@febriaprizal2517             1
@opanboc5038                  1
Name: author, Length: 36170, dtype: int64 

Jumlah Frekuensi/Kemuncul

Seperti yang sudah diinformasikan sebelumnya bahwasanya pada kolom "channel_id" dan "channel_name" memiliki 3 data unik yang merupakan sumber/channel/tempat dimana data-data komentar diambil menggunakan YouTube Data API v3 dari Google Cloud Platform Console berdasarkan topik utama dalam penelitian kali ini yaitu debat Calon Presiden -  Calon Wakil Presiden Republik Indonesia tahun 2024 yang ditayangkan oleh channel Najwa Shihab, KOMPASTV dan KPU RI. Channel Najwa Shihab memiliki ID channel YouTube yaitu UCo8h2TY_uBkAVUIc14m_KCA, channel YouTube KOMPASTV memiliki ID channel UCVdSz86o9q2cQKvAQ5QhOYg dan terakhir adalah channel KPU RI yang memiliki ID channel yaitu UC5BMIWZe9isJXLZZWPWvBlg. Channel Najwa Shihab dengan ID channel yakni UCo8h2TY_uBkAVUIc14m_KCA memiliki jumlah frekuensi/kemunculan nilai sebanyak 22.999 data di dalam masing-masing kedua kolom tersebut yaitu kolom "channel_id" dan "channel_name". Kemudian channel KPU RI dengan ID channel yakni UC5BMIWZe9isJXLZZWPWvBlg memiliki frekuensi kemunculan nilai data sebanyak 12.740 kali di kedua kolom tersebut. Terakhir adalah channel KOMPASTV dengan ID channel UCVdSz86o9q2cQKvAQ5QhOYg yang memiliki jumlah kemunculan nilai data sebanyak 12.500, baik di dalam kolom "channel_id" dan kolom "channel_name".

Insight selanjutnya yang dapat kita ambil berdasarkan output mengenai jumlah frekuensi/kemunculan data dari beberapa kolom adalah ternyata nilai netral pada kolom "sentiment_label" merupakan nilai yang paling banyak muncul di dalam kolom "sentiment_label". Jumlah kemunculan nilai netral pada kolom "sentiment_label" adalah 24.587 kali sedangkan nilai positif hanya 18.178 kali dan nilai negatif sebanyak 5.474 kali. Untuk sementara dari insight ini kita asumsikan bahwasanya mayoritas akun yang berkomentar di dalam video tayangan debat capres - cawapres tahun 2024 yang dikumpulkan dari tiga channel YouTube politik seperti Najwa Shihab, KOMPASTV dan KPU RI memiliki pendapat dan pilihan yang netral jika melihat dari susunan kalimat komentar yang telah dibersihkan dan diproses menggunakan metode Natural Language Processing (NLP).

In [64]:
# Menampilkan jumlah data yang terduplikasi pada variabel DataFrame "youtube_comments_df"
print("Jumlah Data Duplikat pada Variabel DataFrame 'youtube_comments_df' adalah:", youtube_comments_df.duplicated().sum())

Jumlah Data Duplikat pada Variabel DataFrame 'youtube_comments_df' adalah: 0


Berdasarkan hasil diatas dapat diketahui jika variabel DataFrame tidak memiliki data duplikat di dalamnya. Perlu diketahui dan diingat kembali bahwasanya data duplikat merupakan sebuah pola data yang memiliki kesamaan nilai dalam baris yang berbeda sehingga jika salah satu data dihapus/dihilangkan dalam suatu tabel maka data tersebut tidak akan sepenuhnya hilang. Tetapi walaupun begitu dalam penelitian data science tidak mengizinkan adanya duplikasi data atau redudansi data sehingga jika terdapat data duplikat maka data tersebut harus dihilangkan dari baris data supaya tidak menghasilkan analisis yang bias.

In [65]:
# Menampilkan jumlah nilai data yang hilang dari masing-masing kolom dari variabel DataFrame "youtube_comments_df"
youtube_comments_df.isnull().sum()

video_id                 0
channel_id               0
channel_name             0
comment_id               0
author                   0
dirty_text_comment       0
clean_text_comment    1203
like_count               0
total_reply              0
published_date           0
updated_date             0
subjectivity             0
polarity                 0
sentiment_label          0
dtype: int64

Berdasarkan hasil output diatas menunjukkan jika pada kolom "clean_text_comment" memiliki 1.203 baris data yang tidak memiliki nilai sehingga baris-baris tersebut diidentifikasi sebagai nilai NaN atau Not a Number. Pada tahapan sebelumnya, penelitian ini telah berhasil mengubah kolom "author" dan kolom "channel_url" yang sebelumnya memiliki nilai NaN menjadi nilai String yaitu Unknown. Namun khusus untuk mengatasi nilai NaN yang terdapat pada kolom "clean_text_comment", penelitian ini akan menghapus 1.203 data yang memiliki nilai NaN di dalamnya.

Asumsi saya kenapa terdapat 1.203 adalah karena kolom "clean_text_comment" merupakan sebuah kolom yang menyimpan kalimat-kalimat komentar yang telah dilakukan proses text processing seperti tokenization, stemming, stopwords removal, removal punctuations dan stemming dan mungkin saja 1.203 komentar tersebut sebelumnya ditulis dengan komentar yang mengandung unsur angka, emoticon dan kata-kata yang termasuk ke dalam list stopwords removal sehingga secara otomatis akan terhapus. Perlu diketahui dan diingat kembali bahwa stopwords removal di dalam tahapan text processing bertujuan untuk menghilangkan/menghapus kata yang tidak memiliki makna dan arti menurut Bahasa Indonesia yang dibuat di dalam modul NLTK.Corpus dengan fungsi stopwords. Sedangkan removal punctuation merupakan tahapan di dalam text processing yang bertujuan untuk membersihkan susunan kalimat dari berbagai macam simbol-simbol, karakter spesial maupun emoticon.

Maka dari itu saya akan menampilkan 25 baris data NaN yang terdapat di dalam kolom "clean_text_comment" pada variabel DataFrame "youtube_comments_df".

In [66]:
"""
Menampilkan 25 baris data yang memiliki nilai NaN pada
kolom "clean_text_comment" dari variabel DataFrame "youtube_comments_df"
"""
youtube_comments_df_null_rows = youtube_comments_df.isnull().any(axis = 1)
null_rows = youtube_comments_df[youtube_comments_df_null_rows]

null_rows.head(25)

video_id                channel_id                  channel_name  \
2083  yNO0YS846kU  UCVdSz86o9q2cQKvAQ5QhOYg        KPU RI YouTube Channel   
2394  tEbRB5xxOPI  UCo8h2TY_uBkAVUIc14m_KCA  Najwa Shihab YouTube Channel   
2428  8J66JxvmEzo  UCVdSz86o9q2cQKvAQ5QhOYg        KPU RI YouTube Channel   
2916  8J66JxvmEzo  UCVdSz86o9q2cQKvAQ5QhOYg        KPU RI YouTube Channel   
2939  8J66JxvmEzo  UCVdSz86o9q2cQKvAQ5QhOYg        KPU RI YouTube Channel   
3365  8J66JxvmEzo  UCVdSz86o9q2cQKvAQ5QhOYg        KPU RI YouTube Channel   
3668  yNO0YS846kU  UCVdSz86o9q2cQKvAQ5QhOYg        KPU RI YouTube Channel   
3985  YzC828FYrwM  UCVdSz86o9q2cQKvAQ5QhOYg        KPU RI YouTube Channel   
4072  8J66JxvmEzo  UCVdSz86o9q2cQKvAQ5QhOYg        KPU RI YouTube Channel   
4365  8J66JxvmEzo  UCVdSz86o9q2cQKvAQ5QhOYg        KPU RI YouTube Channel   
4366  8J66JxvmEzo  UCVdSz86o9q2cQKvAQ5QhOYg        KPU RI YouTube Channel   
4407  8J66JxvmEzo  UCVdSz86o9q2cQKvAQ5QhOYg        KPU RI YouTube Channel   
4485  YzC828FYrwM  UCVdSz86o9q2cQKvAQ5QhOYg        KPU RI YouTube Channel   
4488  8J66JxvmEzo  UCVdSz86o9q2cQKvAQ5QhOYg        KPU RI YouTube Channel   
4505  YzC828FYrwM  UCVdSz86o9q2cQKvAQ5QhOYg        KPU RI YouTube Channel   
4757  JFl2fOTBNIE  UCo8h2TY_uBkAVUIc14m_KCA  Najwa Shihab YouTube Channel   
4972  JFl2fOTBNIE  UCo8h2TY_uBkAVUIc14m_KCA  Najwa Shihab YouTube Channel   
5026  tEbRB5xxOPI  UCo8h2TY_uBkAVUIc14m_KCA  Najwa Shihab YouTube Channel   
5476  YzC828FYrwM  UCVdSz86o9q2cQKvAQ5QhOYg        KPU RI YouTube Channel   
5479  YzC828FYrwM  UCVdSz86o9q2cQKvAQ5QhOYg        KPU RI YouTube Channel   
5485  YzC828FYrwM  UCVdSz86o9q2cQKvAQ5QhOYg        KPU RI YouTube Channel   
5598  YzC828FYrwM  UCVdSz86o9q2cQKvAQ5QhOYg        KPU RI YouTube Channel   
5601  YzC828FYrwM  UCVdSz86o9q2cQKvAQ5QhOYg        KPU RI YouTube Channel   
5829  Sbjsg3AFx00  UCo8h2TY_uBkAVUIc14m_KCA  Najwa Shihab YouTube Channel   
6084  JFl2fOTBNIE  UCo8h2TY_uBkAVUIc14m_KCA  Najwa Shihab YouTube Channel   

                      comment_id                        author  \
2083  UgwAl-TcRXbfPGh-6d54AaABAg                @siapakah_saya   
2394  UgwcvlwzFm9WwBDcoFN4AaABAg                  @FerryKur666   
2428  UgwzkqWxAeXqc3agLVJ4AaABAg              @L.Am.AnRony1002   
2916  UgwDg_vJ4gldo1gy_HR4AaABAg                @theleggend925   
2939  UgzVsY1W2uZTinSCdFt4AaABAg       @dewiekaprawitarani3700   
3365  UgxoXIs_cj6qWgNTkHt4AaABAg              @user-it7ji7us2r   
3668  Ugw3qwBZFqQftp07Twd4AaABAg                @tiamonica6322   
3985  UgywljMrhhrv7IU-eEN4AaABAg             @desaguyangan4910   
4072  UgzjpRvb5zESsicwGBl4AaABAg              @user-md8lq9xu8l   
4365  UgwbWuOu5SPzepfFMI54AaABAg                 @evanhady4831   
4366  UgziEZy5fmLY0tv6lC94AaABAg                 @makealfa4383   
4407  Ugxpwk8MiF-0GS9VZId4AaABAg                    @GassDetri   
4485  UgzTrOnOmlVJI-deNGJ4AaABAg                  @suebebs6791   
4488  UgwsD5_lQNVd2iOOhcl4AaABAg  @jaringanbintangindonesia786   
4505  UgzB5V2Qa0o_hry4lmJ4AaABAg                 @mustaram7282   
4757  UgxxNGIdTPJuhfLwpQ14AaABAg             @dendievygeboy348   
4972  UgwD0SkKg10wDZBRIkt4AaABAg                @SMProjectNews   
5026  UgzpLiSiQIhkkFIHZUB4AaABAg                     @ED-bb8rk   
5476  Ugy-GJgOzhgSimV26Y14AaABAg             @sutarnogayoh5482   
5479  Ugz0RTbNFWHXz0_T2BB4AaABAg              @dahliazahra8067   
5485  UgwwvLwb9xhCskj6ZF94AaABAg              @user-nv7qv6zk5k   
5598  UgxLGelZvcZ0m5eykjp4AaABAg              @nursitisiti3522   
5601  UgzK-aCbmagcc_boV554AaABAg               @afrizalspd5457   
5829  Ugy2z-aOJAeKf5HRvhV4AaABAg               @MuhamadHarisma   
6084  UgxaDzOKtyC37gQW2pZ4AaABAg                @irma_erviana7   

                   dirty_text_comment clean_text_comment  like_count  \
2083                                0                NaN          19   
2394        Makin Yakin Sama Pak 03 ❤                NaN          15   
2428                             ✌️✌️                NaN          15   
2

Hasil diatas menunjukkan jika nilai NaN yang menjadi representasi missing value pada kolom "clean_text_comment" sebelumnya merupakan komentar yang dibuat menggunakan emoticon, angka atau kata-kata yang termasuk ke dalam list stopwords removal. Hal tersebut bisa dilihat pada indeks data ke-2916 pada kolom "dirty_text_comment", komentar yang dibuat oleh akun channel Youtube bernama @theleggend925 dibuat menggunakan emoticon sebagai bentuk dukungannya kepada pasangan paslon nomor urut 2. Karena emoticon termasuk ke dalam removal punctuations yang telah didefinisikan pada fungsi cleansing_text, maka komentar tersebut otomatis akan terhapus pada indeks data ke-2916 sehingga yang tersimpan di dalam kolom "clean_text_comment" adalah nilai NaN sebagai format jika tidak ada nilai data.

Maka dari itu pada penelitian kali ini saya akan menghapus setiap baris/indeks data yang memiliki nilai NaN (Not a Number) sehingga nantinya jumlah data yang akan dimiliki oleh variabel DataFrame "youtube_comments_df" akan berkurang karena tidak adanya indeks data dengan nilai NaN/missing value seperti pada indeks data ke-2916.

In [67]:
"""
Menghapus missing values (nilai yang hilang) dari beberapa indeks/baris
data yang terdapat di dalam kolom "clean_text_comment" yang direpresentasikan sebagai NaN (Not a Number)
"""
youtube_comments_df.dropna(axis=0, inplace=True)

# Menampilkan sekaligus mengecek jumlah missing values di masing-masing kolom dari variabel DataFrame "youtube_comments_df"
youtube_comments_df.isnull().sum()

video_id              0
channel_id            0
channel_name          0
comment_id            0
author                0
dirty_text_comment    0
clean_text_comment    0
like_count            0
total_reply           0
published_date        0
updated_date          0
subjectivity          0
polarity              0
sentiment_label       0
dtype: int64

In [68]:
3

(47036, 14)

In [69]:
"""
Menampilkan beberapa informasi yang dimiliki oleh variabel DataFrame "youtube_comments_df"
seperti tipe data dari masing-masing kolom, jumlah baris data atau kolom yang dimiliki oleh variabel DataFrame,
nama-nama kolom yang dimiliki oleh DataFrame dan apakah masing-masing kolom masing terdapat missing value/tidak
"""
youtube_comments_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47036 entries, 0 to 48238
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   video_id            47036 non-null  object        
 1   channel_id          47036 non-null  object        
 2   channel_name        47036 non-null  object        
 3   comment_id          47036 non-null  object        
 4   author              47036 non-null  object        
 5   dirty_text_comment  47036 non-null  object        
 6   clean_text_comment  47036 non-null  object        
 7   like_count          47036 non-null  int64         
 8   total_reply         47036 non-null  int64         
 9   published_date      47036 non-null  datetime64[ns]
 10  updated_date        47036 non-null  datetime64[ns]
 11  subjectivity        47036 non-null  float64       
 12  polarity            47036 non-null  float64       
 13  sentiment_label     47036 non-null  object    

Berdasarkan output yang didapatkan dari fungsi "info()" diatas yang menunjukkan beberapa informasi dasar yang terdapat di dalam DataFrame "youtube_comments_df", kita dapat mengetahui beberapa poin informasi dari DataFrame tersebut diantaranya adalah sebagai berikut:

1. Variabel DataFrame "youtube_comments_df" merupakan sebuah objek yang bersifat sebuah tabel tabular yang di dalamnya terdapat susunan kolom, baris/row dan cell yang disebut sebagai DataFrame. DataFrame "youtube_comments_df" merupakan sebuah data yang tersimpan dengan format tabular dari hasil proses penggabungan beberapa data-data komentar yang didapatkan dari video YouTube tentang debat calon Presiden - Wakil Presiden Republik Indonesia tahun 2024 babak pertama hingga babak kelima yang ditayangkan oleh channel YouTube Najwa Shihab, KOMPASTV dan KPU RI.

2. Variabel DataFrame "youtube_comments_df" saat ini memiliki jumlah data/baris sebanyak 47.036 data yang terdiri dari kalimat-kalimat komentar yang dibuat dan dikirimkan oleh para channel akun yang menonton siaran debat, nama channel akun yang berkomentar, tanggal dan waktu dari komentar tersebut dibuat, dll. Sebelumnya variabel DataFrame "youtube_comments_df" memiliki jumlah data sebanyak 48.239 namun karena 1.203 data yang terdapat di dalam kolom "clean_text_comment" merupakan nilai NaN alias baris data yang tidak memiliki nilai, maka seluruh nilai NaN yang terdapat di dalam kolom "clean_text_comment" dihilangkan dari daftar baris data/indeks pada DataFrame "youtube_comments_df".

3. Saat ini variabel DataFrame "youtube_comments_df" memiliki 14 kolom yang diantaranya adalah kolom "video_id", "channel_id", "channel_name", "comment_id", "author", "dirty_text_comment", "clean_text_comment", "like_count", "total_reply", "published_date", "updated_date", "subjectivity", "polarity", dan terakhir adalah kolom "sentiment_label".

4. Masing-masing kolom yang ada di dalam variabel DataFrame "youtube_comments_df" sudah tidak memiliki nilai NaN (Not a Number) atau nilai yang hilang (missing value) sehingga dapat disimpulkan bahwa 14 kolom yang dimiliki oleh variabel DataFrame "youtube_comments_df" dan setiap baris indeksnya data yang dimiliki oleh 14 kolom, saat ini sudah tidak ada satupun baris data pada indeks yang tidak memiliki nilai/data dengan representasi NaN seperti pada variabel null_rows yang masih memiliki missing value di dalam kolom "clean_text_comment".

5. Pada variabel DataFrame "youtube_comments_df" memiliki 4 jenis tipe data yang berbeda. Tipe data yang pertama adalah tipe data Integer untuk setiap kolom yang memiliki nilai data bersifat yang angka atau numerik. Data numerik/data angka dibedakan menjadi 2 jenis bilangan angka yaitu bilangan bulat dan bilangan desimal. Angka bilangan bulat direpresentasikan sebagai tipe data Integer dan angka bilangan desimal direpresentasikan sebagai tipe data Float. Karena pada DataFrame "youtube_comments_df" hanya memiliki 2 kolom saja yang bertipe data Integer, yaitu kolom "like_count" dan "total_reply", di mana kedua kolom tersebut memiliki nilai angka bilangan bulat, maka kolom tersebut bertipe data Integer bukan Float. Tipe data yang kedua adalah tipe data Datetime dan hanya 2 kolom saja yang memiliki tipe data Datetime yaitu kolom "published_date" dan "updated_date". Tipe data yang ketiga yang dimiliki oleh variabel DataFrame "youtube_comments_df" adalah tipe data Object/String. Tipe data Object/String adalah representasi dari nilai-nilai data yang dibuat dengan susunan kata dan kalimat, simbol-simbol maupun karakter spesial yang dimiliki oleh suatu kolom di dalam DataFrame. Tipe data terakhir yang terdapat di dalam variabel DataFrame "youtube_comments_df" adalah tipe data Float dan hanya 2 kolom yang memiliki tipe data Float yaitu kolom "subjectivity" dan kolom "polarity". Secara jumlah, tipe data Object/String pada variabel DataFrame "youtube_comments_df" dimiliki oleh 8 kolom (terbanyak diantara tipe data Integer, Datetime atau Float) yaitu kolom "video_id", "channel_id", "channel_name", "comment_id", "author", "dirty_text_comment", "clean_text_comment" dan kolom "sentiment_label".